# ATEK Data Proprocessing Example

### 1. Load the example adt data

In [5]:
import os

from atek.data_preprocess.sample_builders.obb_sample_builder import ObbSampleBuilder
from atek.data_preprocess.wds_writer import AtekWdsWriter

from omegaconf import OmegaConf

example_adt_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/adt_data_example/Apartment_release_clean_seq134/1WM103600M1292_optitrack_release_clean_seq134"
example_ase_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0"
config_path = os.path.join("/home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/cubercnn_preprocess_adt_config.yaml")

conf = OmegaConf.load(config_path)

### 2. Sample builder example

In [10]:
sample_builder = ObbSampleBuilder(
            conf=conf.processors,
            vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
            mps_files={
                "mps_closedloop_traj_file": os.path.join(
                    example_adt_data_dir, "aria_trajectory.csv"
                ),
            },
            gt_files={
                "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),
                "obb3_traj_file": os.path.join(
                    example_adt_data_dir, "scene_objects.csv"
                ),
                "obb2_file": os.path.join(example_adt_data_dir, "2d_bounding_box.csv"),
                "instance_json_file": os.path.join(
                    example_adt_data_dir, "instances.json"
                ),
            },
        )

# Print the frame dataclass keys
sample = sample_builder.get_sample_by_timestamp_ns(14_594_232_547_562)
print(sample.to_flatten_dict().keys())

Loaded #closed loop trajectory poses records: 2816
dict_keys(['MFCD#camera-rgb+images_0.jpeg', 'MFCD#camera-rgb+capture_timestamps_ns.pth', 'MFCD#camera-rgb+frame_ids.pth', 'MFCD#camera-rgb+exposure_durations_s.pth', 'MFCD#camera-rgb+gains.pth', 'MFCD#camera-rgb+camera_label.txt', 'MFCD#camera-rgb+T_Device_Camera.pth', 'MFCD#camera-rgb+camera_model_name.txt', 'MFCD#camera-rgb+projection_params.pth', 'MFCD#camera-rgb+origin_camera_label.txt', 'MTD#Ts_World_Device.pth', 'MTD#capture_timestamps_ns.pth', 'MTD#gravity_in_world.pth', 'GtData.json'])


[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/adt_data_example/Apartment_release_clean_seq134/1WM103600M1292_optitrack_release_clean_seq134/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Timecode stream found: 285-2
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated


### 2. Write wds files

In [12]:
from atek.data_preprocess.wds_writer import (
    AtekWdsWriter,
)
from atek.data_preprocess.subsampling_lib.temporal_subsampler import CameraTemporalSubsampler

subsampler = CameraTemporalSubsampler(vrs_file = os.path.join(example_adt_data_dir, "video.vrs"), conf = conf.camera_temporal_subsampler)

conf.wds_writer.prefix_string = "test"
atek_wds_writer = AtekWdsWriter(output_path = "/home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt", conf = conf.wds_writer)

for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)
    
    for t in timestamps_ns:
        sample = sample_builder.get_sample_by_timestamp_ns(t)
        if sample is not None:
            atek_wds_writer.add_sample(data_sample = sample)

atek_wds_writer.close()


[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/adt_data_example/Apartment_release_clean_seq134/1WM103600M1292_optitrack_release_clean_seq134/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Timecode stream found: 285-2
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated
Querying MPS traj for 14579434921812 has returned None, skipping this sample.
Querying MPS traj for 14579634891650 has returned None, skipping this 

# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0000.tar 0 0.0 GB 0
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0001.tar 32 0.0 GB 32
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0002.tar 32 0.0 GB 64
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0003.tar 32 0.0 GB 96
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0004.tar 32 0.0 GB 128
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0005.tar 32 0.0 GB 160
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_cubercnn_adt/shards-0006.tar 32 0.0 GB 192
# writing /home/louy/Calibration_data_link/Atek/2024_05_28_CubeRcnnTest/wds_output/test_c

Querying MPS traj for 14681818533475 has returned None, skipping this sample.
Querying MPS traj for 14682018501062 has returned None, skipping this sample.
Querying MPS traj for 14682218472812 has returned None, skipping this sample.
Querying MPS traj for 14682418433812 has returned None, skipping this sample.
Querying MPS traj for 14682618401475 has returned None, skipping this sample.
Querying MPS traj for 14682818369187 has returned None, skipping this sample.
Querying MPS traj for 14683018336850 has returned None, skipping this sample.
Querying MPS traj for 14683218312725 has returned None, skipping this sample.
Querying MPS traj for 14683418283725 has returned None, skipping this sample.
Querying MPS traj for 14683618251437 has returned None, skipping this sample.
Querying MPS traj for 14683818222812 has returned None, skipping this sample.
Querying MPS traj for 14684018186062 has returned None, skipping this sample.
Querying MPS traj for 14684218149900 has returned None, skipping